# Malware Simulation Demo / Happy Holidays

. | .
- | - 
![alt](gift.png) | ![alt](Element-AI-Symbol-RGB.jpg)

This notebook shows how to run a simple ITSim simulation and collect its telemetry data from a datastore server. Then it quickly overlooks the generated data by showing the telemetry records and plotting the chronological graphs of events.


## Step 1:  Launch a Datastore Server to collect simulation data

This server collects ITSim telemetry and logs over a REST API and archives them into a Database (SQLite db). This server is running throughout the simulations. It could be used to collect data from simulations running simultaneously on multiple machines.    

In [ ]:
import os

DB_FILE = "malware_simulation_1.sqlite"
HOSTNAME = "localhost"
SERVER_PORT = "5000"

os.system(f'python ../bin/itsim_serve_datastore.py --sqlite_file {DB_FILE} --host {HOSTNAME} --port {SERVER_PORT} &')

## Step 2:  Run ITSim Simulations

### Define Simulation Parameters

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

def set_simulation_parameters(sim_runs, network_topology,nb_endpoints, backdoor_random_start):
    return {'sim_runs':sim_runs,
            'network_topology':network_topology,
            'nb_endpoints':nb_endpoints,
            'backdoor_random_start':backdoor_random_start}
            
style={'description_width': 'initial'}

sim_runs_widget = widgets.IntSlider(value=4,
                                    min=1,
                                    max=10,
                                    description='Simulations:',
                                    style=style)

network_topology_widget = widgets.Dropdown(options=['Flat + Internet'],
                                           value='Flat + Internet',
                                           description='Topology:',
                                           style=style)

nb_endpoints_widget = widgets.IntSlider(value=20,
                                        min=2,
                                        max=65000,
                                        description='Endpoints:',
                                        style=style)

backdoor_random_start_widget = widgets.IntRangeSlider(value=[15, 45],
                                                   min=1,
                                                   max=100,
                                                   description='Backdoor Uni(a,b):',
                                                   style=style)

sim = interactive(set_simulation_parameters, 
                sim_runs=sim_runs_widget, 
                network_topology=network_topology_widget,
                nb_endpoints=nb_endpoints_widget,
                backdoor_random_start = backdoor_random_start_widget)
display(sim)


### Setting up a simulation runner

In [ ]:
def run_simulations(sim_cfg, sim_fct):
    print(f"Running {sim_cfg.result['sim_runs']} simulations based on the following configuration:")
    print(f"\t- Network Topology: \t\t\t\t\t{sim_cfg.result['network_topology']}")
    print(f"\t- Number of servers: \t\t\t\t\t1")
    print(f"\t- Number of Endpoints: \t\t\t\t\t{sim_cfg.result['nb_endpoints']}")
    print(f"\t- Random backdoor callback based on distribution: \tUniform({sim_cfg.result['backdoor_random_start'][0]},{sim_cfg.result['backdoor_random_start'][1]})")

    bar = widgets.IntProgress(value=0,
                              min=0,
                              max=sim_cfg.result['sim_runs'],
                              step=1,
                              description='Progress:',
                              bar_style='info', 
                              orientation='horizontal')
    display(bar)

    for _ in range(sim_cfg.result['sim_runs']):
        sim_fct
        bar.value+=1 

### Define the ITSim Simulation

In [ ]:
from greensim.random import expo, constant
from itsim import malware
from itsim.datastore.datastore import DatastoreClientFactory
from itsim.machine.endpoint import Endpoint
from itsim.network.link import Link
from itsim.simulator import Simulator, advance
from itsim.software.context import Context
from itsim.types import as_cidr, Protocol
from itsim.units import MbPS, MS, S

In [ ]:
PORT = 5678
CIDR = as_cidr("192.168.1.0/24")
NUM_CLIENTS = 5
DELAY_PING = expo(2.0 * S)

def serve_pongs(context: Context) -> None:
    with context.node.bind(Protocol.UDP, PORT) as socket:
        while True:
            packet = socket.recv()
            socket.send(packet.source, 4, {"what": "pong"})


def ask_ping(context: Context) -> None:
    advance(next(DELAY_PING))
    with context.node.bind(Protocol.UDP) as socket:
        socket.send((CIDR.network_address + 1, PORT), 4, {"what": "ping"})
        socket.recv()


@malware
def backdoor(context: Context) -> None:
    advance(1.0 * S)
    ask_ping(context)


def control(sim, pings):
    advance(10000)
    for proc_pinger in pings:
        proc_pinger.wait(NUM_CLIENTS * S)
    sim.stop()


def test_endpoint_telemetry():
    with Simulator() as sim:
        DatastoreClientFactory().sim_uuid = sim.uuid
        link = Link(CIDR, expo(10 * MS), constant(100 * MbPS))
        server = Endpoint().connected_to_static(link, 1)
        server.run_proc(sim, serve_pongs)

        endpoints = [Endpoint().connected_to_static(link, 100 + n) for n in range(NUM_CLIENTS)]
        pings = [endpoint.run_proc(sim, ask_ping) for endpoint in endpoints]
        endpoints[-1].run_proc(sim, backdoor)

        sim.add(control, sim, pings)
        sim.run()
        return sim.uuid
    



In [ ]:
run_simulations(sim, test_endpoint_telemetry)

## Step 3: Shutting down the Datastore Server
We're done with the datastore server, we can close it. 

In [ ]:
import requests

try: 
    response = requests.post(f'http://{HOSTNAME}:{SERVER_PORT}/stop')
    print("Server properly shutdown")
except:
    print("Can't reach the server to shut it down")

## Step 4: Retrieve Simulation Telemetry

Now that the simulations ran to completion, we can access the data collected by the datastore (SQLite database).

The datastore is storing telemetry events as JSON strings into a SQLite database (for the convenience of quick prototyping). This is a quick preview of what the data looks like using Pandas. 

In [ ]:
import sqlite3
import pandas as pd
import json

conn = sqlite3.connect(DB_FILE)
df = pd.read_sql_query("SELECT * FROM network_event;", conn)
df

From this dataframe, we can load the json data and list the fields of the telemetry events: 

In [ ]:
simulations = df.sim_uuid.unique()
print(f"Telemetry events for {len(simulations)} simulations")
df_json = []

simulations_data = {}
for sim in simulations:
    df2 = df.loc[df['sim_uuid'] == sim]
    df2 = df2.set_index("timestamp", drop = False)
    df1 = df2.loc[:, 'json'].to_frame()
    entry_list = []
    for _, row in df1.iterrows():
        entry_list.append(json.loads(row.json))
    simulations_data[sim] = entry_list

for sim_uuid, telemetry_list in simulations_data.items():
    print(f"\nSimulation {sim_uuid}\n")
    for telemetry in telemetry_list:
        print(f"Telemetry {telemetry['uuid']}")
        for key, value in telemetry.items():
            print(f"\t{key}: {value}")


## Step 5: Plot Telemetry Data
Here is a "quick and dirty approach" to plot telemetry events chronologically: 

In [ ]:
import ast
import random
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
print(__version__) # requires version >= 1.9.0

def plot_simulation_telemetry_events(conn, sim_uuid):
    node_uuid_idx = 0
    node_timestamp_idx = 1
    node_simulation_idx = 2
    node_json_idx = 3
    df = []

    print(f"Simulation: {sim_uuid}")
    with conn:
        cursor = conn.cursor()
        cursor.execute(f'SELECT * FROM network_event WHERE "sim_uuid"="{sim_uuid}"')
        all_entries = cursor.fetchall()    
        cnt = 0
        colors = []
        net_events_post_proc = []

        for network_event in all_entries:
            data = ast.literal_eval(network_event[node_json_idx])      
            uuid_node = data["uuid_node"]
            event_info = {"uuid_node": uuid_node,"start":"","stop":""}
            net_events_post_proc.append(event_info)

        for network_event in all_entries:
            data = ast.literal_eval(network_event[node_json_idx])      
            uuid_node = data["uuid_node"]   
            event_dict = next(item for item in net_events_post_proc if item["uuid_node"] == uuid_node)
            if event_dict["start"]=="":
                event_dict["start"]=network_event[node_timestamp_idx]
            else:
                event_dict["stop"]=network_event[node_timestamp_idx]

        for network_event_dict in net_events_post_proc:
            if network_event_dict["start"] is not "" and network_event_dict["stop"] is not "":
                colors.append((random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1)))
                df.append(dict(Task=network_event_dict["uuid_node"], Start=network_event_dict["start"], Finish=network_event_dict["stop"], Event_Name="connection"))
        init_notebook_mode(connected=True)
        fig = ff.create_gantt(df, title="ITsim Network Events", colors=colors, index_col='Event_Name', showgrid_x=True, showgrid_y=True, show_colorbar=True, group_tasks=True)
        iplot(fig)


We're now calling this function for every simulations ran:

In [ ]:
for sim_uuid in simulations:
    plot_simulation_telemetry_events(conn, sim_uuid)

# THANKS!!
